In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import List, Tuple
from RF_EUR import zcb_price_generator

# Notebook display prefs (optional)
pd.set_option("display.float_format", lambda x: f"{x:,.6f}")



We define parameters and calculate our outcome for single tie-in path given active portfolio return and zcb prices

In [28]:
@dataclass
# Define parameters for the tie-in strategy
class TieInConfig:
    initial_wealth: float = 0
    L_target: float = 1.25   
    L_trigger: float = 1.30


@dataclass
# Zero coupon bond parameters
class PathResult:
    final_N: float
    initial_N: float
    # two different ways to measure accrual
    accrual_abs: float
    accrual_pct: float
    history: pd.DataFrame


# Simulate a single path of the tie-in strategy
# Important to note cfg is not needed
def simulate_tie_in_path(
    active_returns: np.ndarray,       # has dimension (T) (monthly discrete returns of Active)
    zcb_prices: np.ndarray,           # starts at 0 so has dimension (T+1) (rolling ZCB prices, including month 0 and maturity)
    contributions: np.ndarray = None, # has dimension (T) (monthly contributions, if any) - VÆR OPMÆRKSOM PÅ INITIAL WEALTH ER 100 SÅ C0 ER 0.
    cfg: TieInConfig | None = None,   # if we want to change parameters in TieInFonfig we can: simulate_tie_in_path(active, zcb, TieInConfig(L_target=1.20))
) -> pd.DataFrame:
    if cfg is None:
        cfg = TieInConfig()
    
    T = len(active_returns)
    assert len(zcb_prices) == T + 1, "zcb_prices must have length T+1 (including month 0 and maturity)."

    if contributions is None:
        contributions = np.zeros(T + 1, dtype=float)
    contributions = np.asarray(contributions, dtype=float)
    assert len(contributions) == T + 1, "contributions must have length T+1 (including month 0)."

    W0 = cfg.initial_wealth + contributions[0]            # We can both set initial wealt and contributions
    MV_R = W0 / cfg.L_target                              # start exactly at target funded ratio
    MV_A = W0 - MV_R                                      # Rest in active
    N = MV_R / zcb_prices[0]                              # initial face (units of the ZCB)

    rows = []
    rows.append({
        "month": 0,
        "MV_A": MV_A,
        "MV_R": MV_R,
        "N": N,
        "P": zcb_prices[0],
        "W": MV_A + MV_R,
        "C": contributions[0],
        "c_A": contributions[0]*MV_A/W0, 
        "c_R": contributions[0]*MV_R/W0,
        "L": (MV_A + MV_R) / MV_R,
        "tie_in": False,
    })
    
    # Do the calculations month by month
    for i in range(1, T+1):
        # 1) Active evolves                                     - VI SKAL OGSÅ LIGE VÆRE OPMÆRKSOMME PÅ HVORDAN ACTIVE RETURNS ER GIVET I VALGT STRATEGI
        MV_A = MV_A * (1.0 + active_returns[i-1])              #- HVIS ACTIVE RETURNS ER AGGREGEREDE SKAL DET VÆRE SÅDAN, ELLERS SKAL ÆNDRES
        # 2) Reserve repriced from rolling ZCB
        P_i = zcb_prices[i]
        MV_R = N * P_i
        # Total wealth
        W = MV_A + MV_R

        # Calculate funded ratio
        L = (MV_A + MV_R) / MV_R
        tie_in = L > cfg.L_trigger
        
        # Make relevant adjustments if the funded ratio exceeds the trigger level
        if tie_in:
            # Reset to target: W = L_target * N_new * P_i
            N_new = W / (cfg.L_target * P_i)       # new face (units of the ZCB)
            # new reserve and active
            MV_R = N_new * P_i
            MV_A = W - MV_R
        
        # Calculate contributions based on the value of active/reserve portfolio
        C = contributions[i]        
        c_R = C * MV_R / (W)
        c_A = C * MV_A / (W)
        # 4) Update active and reserve with contributions
        MV_R += c_R
        MV_A += c_A
        # 5) Add contribution of ZCB to guarentee
        N += c_R/P_i
            

        rows.append(dict(month=i, MV_A=MV_A, MV_R=MV_R, c_A = c_A, c_R = c_R, N=N, P=P_i, W=MV_A+MV_R, L=(MV_A+MV_R)/MV_R, tie_in=tie_in))

    history = pd.DataFrame(rows, columns=['month','MV_A','MV_R', "c_A", "c_R",'N','P','W','L','tie_in'])
    return history



## Random active portfolio and using Jakob's ZCB data:


In [ ]:
# --- Synthetic data for a 10-year
np.random.seed(7)
T = 120

# Contributions - VIRKER IKKE LIGE NU
contributions = np.zeros(T+1) + 100

# Random active returns
mu, sigma = 0.006, 0.1
active_returns = np.random.normal(mu, sigma, size= T)

# --- Implement real data ---
df = pd.read_csv('csv_files/risk_free_rates_eur.csv', parse_dates=['TIME_PERIOD'])
rates = df['RISK_FREE_RATE'].to_numpy(dtype=float)


months = np.arange(T + 1)
ttm_years = (T - months) / 12.0
# convert to decimals
zcb_prices = zcb_price_generator(10, 121, start = 0, data = pd.read_csv("csv_files/yield_curve_data.csv"))

# Sanity checks
assert active_returns.shape == (T,)
assert zcb_prices.shape == (T + 1,)

# Run
summary = simulate_tie_in_path(active_returns, zcb_prices, contributions)
print(summary)
print(sum(contributions))

     month         MV_A          MV_R       c_A       c_R             N  \
0        0    20.000000     80.000000 20.000000 80.000000    128.974124   
1        1    46.372476    156.380423 22.871424 77.128576    254.492846   
2        2    66.699174    233.624038 22.209130 77.790870    381.533962   
3        3    89.396440    315.512423 22.078163 77.921837    506.664455   
4        4   100.936455    403.745819 20.000000 80.000000    636.951571   
..     ...          ...           ...       ...       ...           ...   
116    116 2,515.105698 12,348.638862 16.921077 83.078923 12,457.652287   
117    117 2,552.793888 12,461.195081 17.002769 82.997231 12,541.182269   
118    118 2,475.025924 12,572.881976 16.447641 83.552359 12,625.081516   
119    119 2,345.067990 12,687.755914 15.599651 84.400349 12,709.627357   
120    120 2,786.846099 12,791.738401 17.888956 82.111044 12,791.738401   

           P             W        L  tie_in  
0   0.620279    100.000000 1.250000   False  
1   0.6